# 整体调度处理模块

In [5]:
from utils import *
from attention import MultiHeadedAttention
from positionwiseFeedForward import PositionwiseFeedForward
from positional_encoding import PositionalEncoding
from encoder import Encoder, EncoderLayer
from decoder import Decoder, DecoderLayer
from embedding_softmax import Embeddings
import torch.nn.functional as F

In [6]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, input_embed, target_embed, line_softmax):
        """
        Encoder - Decoder架构
        1 encoder：编码器模型(nn.Module)
        2 decoder：解码器模型(nn.Module)
        3 input_embed: embedding后的encoder测的输入数据(nn.Module)
        4 target_embed: embedding后decodeer侧的输入向量(也就是目标向量)(nn.Module)
        5 line_softmax: 模型decoder侧最后的linear --> softmax(nn.Module)
        """
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.input_embed = input_embed
        self.target_embed = target_embed
        self.line_softmax = line_softmax
    
    def forward(self, input_tensor, target_tensor, input_mask, target_mask):
        """
        input_tensor:输入数据的向量结果
        input_mask：输入数据对应的mask掩码
        target同理
        """
        return self.decode(self.encode(input_tensor, input_mask), input_mask, target_tensor, target_mask)
    
    def encode(self, input_tensor, input_mask):
        return self.encoder(self.input_embed(input_tensor), input_mask)
    
    def decode(self, memory, input_mask, target_tensor, target_mask):
        return self.decoder(self.target_embed(target_tensor), memory, input_mask, target_mask)

In [7]:
class LineSoftmax(nn.Module):
    """定义标准的linear --> softmax."""
    def __init__(self, embed_dim, vocab_size):
        """
        embed_dim: 词向量embedding后的维度
        input_size： 词向量的尺寸
        """
        super(LineSoftmax, self).__init__()
        self.line = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        return F.log_softmax(self.line(x), dim=-1)

# 构建整体模型

In [8]:
def make_model(input_vocab_size, target_vocab_size, N=6, embed_dim=512, encode_dim=2048, h=8, dropout=0.1):
    """
    input_vocab_size:输入词向量的尺寸
    N：编码解码层的重复次数(N个Encoder/Decoder block)
    encode_dim:编码器内层维度
    h： 'Scaled Dot Product Attention'，使用的次数
    dropout： 丢弃机制
    """
    c = copy.deepcopy
    # 实例化多头，前馈层，位置编码
    attention = MultiHeadedAttention(h, embed_dim)
    ff = PositionwiseFeedForward(embed_dim, dropout)
    position = PositionalEncoding(embed_dim, dropout)
    
    """
    根据结构图, 最外层是EncoderDecoder，在EncoderDecoder中，
    分别是编码器层，解码器层，源数据Embedding层和位置编码组成的有序结构，
    目标数据Embedding层和位置编码组成的有序结构，以及类别生成器层.
    在编码器层中有attention子层以及前馈全连接子层，
    在解码器层中有两个attention子层以及前馈全连接层.
    """
    model = EncoderDecoder(
        Encoder(EncoderLayer(embed_dim, c(attention), c(ff), dropout), N),
        Decoder(DecoderLayer(embed_dim, c(attention), c(attention), c(ff), dropout), N),
        nn.Sequential(Embeddings(embed_dim, input_vocab_size), c(position)),
        nn.Sequential(Embeddings(embed_dim, target_vocab_size), c(position)),
        LineSoftmax(embed_dim, target_vocab_size)
    )
    
    # 模型权重初始化使用xavier_uniform
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model